# DBScan Clustering 
Transitioning Emissions Latitude/Longitude records into unsupervised clusters in order to consolidate the 5.96m records into 
a smaller subset of unique clusters with a labeled centerpoint and reference label. 

In [4]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, time
%matplotlib inline

from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
%matplotlib inline

from sqlalchemy import create_engine
import config
import config1

In [5]:
# Expanding number of columns when using .head():
pd.set_option('display.max_columns', 40)

### Loading Data from SQL DB. 

In [6]:
config.db_name  # Checking name of DB. 

'db27bvsdruzh45'

In [7]:
# Create sqlalchemy engine to access database:
engine = create_engine("mysql+mysqlconnector://{user}:{password}@{host}/{dbname}"
                       .format(user=config.db_user,
                               password=config.db_pass,
                               dbname=config.db_main,
                               host=config.db_host))

In [13]:
### Skip this block if reading from local disk:

#Create Query Statments
queryWeather = """
SELECT *
FROM Weather;
"""
queryEmissions = """
SELECT *
FROM Emissions_Data;
"""

In [18]:
### Alternative Load from disk if required. 
emyear20xx = pd.read_csv('../data/emissions_year2003to2015.csv')

In [19]:
emyear20xx.shape

(7254534, 22)

In [20]:
# Reviewing the years across the dataset.
print(emyear20xx['year'].unique())

[2008 2009 2013 2007 2006 2012 2004 2010 2011 2005 2015 2014 2003]


In [21]:
### Remove zero value features noted during initial Emissions data review. 
indexNames = emyear20xx[(emyear20xx.prefire_fuel == 0) & (emyear20xx.consumed_fuel == 0) & (emyear20xx.ECO2 == 0)].index
emyear20xx.drop(indexNames , inplace=True)

In [22]:
emyear20xx.shape

(5960572, 22)

In [7]:
emyear20xx.head()

,id,year,doy,longitude,latitude,grid10k,covertype,fuelcode,area_burned,prefire_fuel,...,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag
0,0,2008,359,-81.0384,25.1958,4536,3,1600,0.0,6220.097576,...,153.981344,4.499455,23.797117,0.023231,0.082115,3,1,81,1,0
1,1,2008,359,-81.0404,25.1984,4536,3,1600,62500.0,6220.097576,...,157.185824,4.593092,24.292355,0.022757,0.080441,3,1,81,2,0
2,2,2008,359,-81.0380,25.1981,4536,3,1600,0.0,6220.097576,...,153.981344,4.499455,23.797117,0.023231,0.082115,3,1,81,1,0
4,4,2008,359,-81.0594,25.2035,4536,3,1600,0.0,6220.097576,...,153.981344,4.499455,23.797117,0.023231,0.082115,3,1,81,1,0
5,5,2008,359,-81.0570,25.2032,4536,3,1600,62500.0,6220.097576,...,157.185824,4.593092,24.292355,0.022757,0.080441,3,1,81,2,0


## Set Year for DBSCan clustering. 

In [114]:
emyear = 2009

In [115]:
### Variable to capture a single year of data for clustering. 
emyearset20xx = emyear20xx[(emyear20xx.year == emyear)]
print('There are {:,} rows'.format(len(emyearset20xx)))

There are 375,841 rows


In [116]:
emyearset20xx.tail(10)

,id,year,doy,longitude,latitude,grid10k,covertype,fuelcode,area_burned,prefire_fuel,...,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag
1008864,496810,2009,240,-120.7744,48.8777,130977,3,1360,62500.0,2673.497587,...,262.016109,14.735944,44.917047,0.168969,0.232857,2,4,15,3,1
1008865,496811,2009,240,-120.7850,48.8782,130977,3,1260,0.0,6989.973251,...,531.699850,29.903119,91.148546,0.380435,0.291163,2,4,15,1,1
1008866,496812,2009,240,-120.7817,48.8787,130977,3,1260,62500.0,6989.973251,...,552.142166,31.052808,94.652943,0.366350,0.280383,2,4,15,2,1
1008867,496813,2009,240,-120.7785,48.8793,130977,3,1260,62500.0,6989.973251,...,552.142166,31.052808,94.652943,0.366350,0.280383,2,4,15,2,1
1008868,496814,2009,240,-120.7826,48.8809,130977,3,1260,62500.0,6989.973251,...,695.238379,39.100625,119.183722,0.290946,0.222674,2,4,15,4,1
1008869,496815,2009,120,-122.1427,48.9261,132351,3,1910,62500.0,9690.845325,...,898.386849,50.525817,154.009174,0.213875,0.330757,2,4,78,3,1
1008870,496816,2009,120,-122.1468,48.9277,132351,3,1200,62500.0,6424.431099,...,646.844460,36.378922,110.887622,0.243417,0.190915,2,4,78,4,1
1008871,496817,2009,120,-122.1436,48.9283,132351,3,1910,62500.0,9690.845325,...,873.389054,49.119926,149.723838,0.219996,0.340224,2,4,78,2,1
1008872,496818,2009,120,-122.1404,48.9289,132351,3,1200,62500.0,6424.431099,...,584.220342,32.856903,100.152059,0.269510,0.211380,2,4,78,3,1
1008873,496819,2009,120,-122.1445,48.9305,132351,3,1910,62500.0,9690.845325,...,919.437623,51.709725,157.617878,0.208978,0.323184,2,4,78,4,1


In [117]:
# Set distance per radian, either KMs or Miles. 
kms_per_radian = 3956
### 3956 for miles, 6371.0088 for kilometers
print(kms_per_radian)

3956


## Compute DBSCAN

 #####   eps is the physical distance from each point that forms its neighborhood
#####    min_samples is the min cluster size, otherwise it's noise - set to 1 so we get no noise

##### Extract the lat, lon columns into a numpy matrix of coordinates, then convert to radians when you call fit, for use by scikit-learn's haversine metric.

In [118]:
coords = emyearset20xx.as_matrix(columns=['latitude', 'longitude'])
epsilon = 1.5 / kms_per_radian

/Users/AlfHaugen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


## Defining the Clustering Functions

In [119]:
emyear

2009

In [120]:
# Set up DBScan model. 
start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

# determine the number of clusters
num_clusters = len(set(cluster_labels))

# Print the outcome
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(emyearset20xx), num_clusters, 100*(1 - float(num_clusters) / len(emyearset20xx)), time.time()-start_time))
print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))


Clustered 375,841 points down to 8,234 clusters, for 97.8% compression in 57.06 seconds
Silhouette coefficient: 0.281


## Results from Clustering:
### Emissions Data from USDA:
##### 2003 - Clustered 317,353 points down to 6,231 clusters, for 98.0% compression in 44.70 seconds
Silhouette coefficient: 0.491
##### 2004 - Clustered 164,634 points down to 6,568 clusters, for 96.0% compression in 21.16 seconds
Silhouette coefficient: 0.605
##### 2005 - Clustered 399,828 points down to 9,010 clusters, for 97.7% compression in 76.10 seconds
Silhouette coefficient: 0.419
##### 2006 West - Clustered 300,874 points down to 2,718 clusters, for 99.14% compression in 43.30 seconds
Silhouette coefficient: 0.359
##### 2006 East - Clustered 190,156 points down to 9,073 clusters, for 95.2% compression in 27.19 seconds
Silhouette coefficient: 0.341
##### 2007 - Clustered 640,039 points down to 8,945 clusters, for 98.6% compression in 110.69 seconds
Silhouette coefficient: 0.269
##### 2008 - Clustered 440,888 points down to 9,264 clusters, for 97.9% compression in 75.88 seconds
Silhouette coefficient: 0.444
##### 2009 - Clustered 375,841 points down to 8,234 clusters, for 97.8% compression in 57.06 seconds
Silhouette coefficient: 0.281
##### 2010 - Clustered 270,486 points down to 8,237 clusters, for 97.0% compression in 30.41 seconds
Silhouette coefficient: 0.512
##### 2011 - Clustered 770,255 points down to 10,554 clusters, for 98.6% compression in 137.69 seconds
Silhouette coefficient: 0.252
##### 2012 - Clustered 667,029 points down to 7,549 clusters, for 98.9% compression in 114.98 seconds
Silhouette coefficient: 0.340
##### 2013 - Clustered 272,406 points down to 5,483 clusters, for 98.0% compression in 34.88 seconds
Silhouette coefficient: 0.528
##### 2014 - Clustered 442,103 points down to 8,059 clusters, for 98.2% compression in 75.23 seconds
Silhouette coefficient: 0.188
##### 2015 - Clustered 483,703 points down to 8,588 clusters, for 98.2% compression in 83.23 seconds
Silhouette coefficient: 0.379

In [121]:
# Turn the clusters into a pandas series, where each record is a cluster of points.
clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])

In [122]:
# Print clusters to see what's in it: ok, looks like all the coordindates for a cluster are within a record row.  
clusters.tail(2)

8232                               [[48.6519, -120.8904]]
8233    [[48.9261, -122.1427], [48.9277, -122.1468], [...
dtype: object

## Capturing individual data points (lat/long) that make up each cluster to save into a DataFrame

In [123]:
ab = list(zip(clusters[0]))
ab = len(ab)
print('Number (len) of Clusters from results = ', len(clusters))
print('Number of cluster datapoints in the first Cluster = ', ab)

Number (len) of Clusters from results =  8234
Number of cluster datapoints in the first Cluster =  359


In [124]:
#clusters[0]  # Curious to review on record. 

In [125]:
### Loop to pull in the cluster points that make up each DBScan Cluster represented in the Series titled 'Clusters'.
def preparecluster(row, clusterseries):
    clusterDF = pd.DataFrame(clusterseries[0]) 
    clusterDF = clusterDF.assign(ClusterNum=row) 
    row = 1
    while row < len(clusterseries):
        clusterdata = (clusterseries[row])
        clusterDFTemp = pd.DataFrame(clusterdata)
        clusterDFTemp = clusterDFTemp.assign(ClusterNum=row) 
        clusterDF = clusterDF.append(clusterDFTemp, ignore_index=True)
        row = row + 1
    return clusterDF

In [126]:
### Viewing results of dataframe consolidation and adding year to the dataframe so we see each Lat/Long, assigned cluster number from DBScan and Year. 
groupedclusters = preparecluster(0,clusters)
groupedclusters = groupedclusters.assign(Year=emyear) 
groupedclusters.tail()

,0,1,ClusterNum,Year
375836,48.9261,-122.1427,8233,2009
375837,48.9277,-122.1468,8233,2009
375838,48.9283,-122.1436,8233,2009
375839,48.9289,-122.1404,8233,2009
375840,48.9305,-122.1445,8233,2009


In [127]:
# Save results so that we may create a consolidated dataframe of all recorded coordinates, their cluster reference and year. 
groupedclusters.to_csv('../data/Emclustermakeup_2009.csv', encoding='utf-8')

## Find the point in each cluster that is closest to its centroid

DBSCAN clusters may be non-convex. This technique just returns one representative point from each cluster. First get the lat,lon coordinates of the cluster's centroid (shapely represents the first coordinate in the tuple as x and the second as y, so lat is x and lon is y here). Then find the member of the cluster with the smallest great circle distance to the centroid.


In [128]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

centermost_points = clusters.map(get_centermost_point)

In [129]:
# unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
latlon = zip(*centermost_points)
print(latlon)

In [130]:
# unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
latitude, longitude = zip(*centermost_points)

In [131]:
print(latitude[1:5], longitude[1:5])

(25.4634, 25.5956, 26.6739, 25.5937) (-81.0448, -81.1593, -97.9236, -80.5409)


In [132]:
# from these lats/lons create a new df of one representative point for each cluster
rep_points = pd.DataFrame({'longitude':longitude, 'latitude':latitude})
rep_points.head()

,longitude,latitude
0,-80.8823,25.2221
1,-81.0448,25.4634
2,-81.1593,25.5956
3,-97.9236,26.6739
4,-80.5409,25.5937


## Pulling related data for new Cluster Points and saving to CSV

In [133]:
# Pull row from original data set where lat/lon match the lat/lon of each row of representative points
# that way we get the full details such as 'covertype, 'fuelcode', etc. from the original dataframe.

rs = rep_points.apply(lambda row: emyearset20xx[(emyearset20xx['latitude']==row['latitude']) & (emyearset20xx['longitude']==row['longitude'])].iloc[0], axis=1)
rs.to_csv('../data/Emissions2006East_DBScan_Clusters.csv', encoding='utf-8')
rs.tail()

,id,year,doy,longitude,latitude,grid10k,covertype,fuelcode,area_burned,prefire_fuel,...,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag
8229,496792.0,2009.0,166.0,-119.6789,48.6127,128679.0,1.0,1.0,0.0,89.108370,...,5.800955,0.222094,0.845282,0.000000,0.000000,2.0,4.0,78.0,1.0,1.0
8230,496796.0,2009.0,247.0,-117.9043,48.9716,129154.0,3.0,1200.0,0.0,6424.431099,...,490.284166,27.573876,84.048714,0.321146,0.251879,2.0,4.0,15.0,1.0,1.0
8231,496802.0,2009.0,186.0,-122.0629,48.3321,129583.0,3.0,1300.0,0.0,11715.036749,...,860.395988,48.389188,147.496455,0.464559,0.275379,2.0,4.0,15.0,1.0,1.0
8232,496803.0,2009.0,210.0,-120.8904,48.6519,130054.0,3.0,1260.0,62500.0,6989.973251,...,629.822967,35.421623,107.969652,0.321165,0.245802,2.0,4.0,78.0,3.0,1.0
8233,496817.0,2009.0,120.0,-122.1436,48.9283,132351.0,3.0,1910.0,62500.0,9690.845325,...,873.389054,49.119926,149.723838,0.219996,0.340224,2.0,4.0,78.0,2.0,1.0
